In [74]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import pandas as pd
from itertools import groupby

In [77]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [78]:
#Считывание данных в DataFrame 
train= pd.read_csv('train_net_2.csv', sep=';', index_col=None)
test= pd.read_csv('test_net_2.csv', sep=';', index_col=None)


In [79]:
train

,team_id,id,Analytical thinking,Systemic thinking,Adaptability,Focus,game_id,Период_1_всего,Период_2_всего,Период_3_всего,...,Категория 4 место стокл,Категория 5 оценка стокл,Категория 5 место стокл,Категория 6 оценка стокл,Категория 6 место стокл,period_1,period_2,period_3,period_4,TestSummAll
0,1440,10884,4,4,4,4,202,7,14,28,...,1.581139,9.772410,0.836660,44.302370,1.414214,Cancellation of bonus for violation of labor d...,"Contextual advertising, Advertising in print m...","Managers qualification, Anti-bureaucratic days...","Size of a reserve stock of WIP, The best aesth...",Cancellation of bonus for violation of labor d...
1,1536,10106,5,4,5,5,211,21,21,12,...,1.414214,12.934450,1.788854,33.611010,1.483240,"Development of productionвЂ™s optimisation, Bo...","Repeat inspection of batch products, Developme...","Optimizing the placement of equipment, Increas...",R&D enhancement of aesthetic and ergonomic qua...,"Development of productionвЂ™s optimisation, Bo..."
2,1742,10438,6,5,5,5,242,62,42,58,...,0.894427,21.799083,0.447214,86.569625,2.121320,"Manufacturing staff qualification, To place pa...",Minimal fixed + % by the results of a period (...,Quality of the purchased materials Raw (carbon...,"Factoring, Quality of the purchased materials ...","Manufacturing staff qualification, To place pa..."
3,1272,10130,5,4,5,5,184,24,24,33,...,1.788854,10.876580,0.547723,51.037241,1.732051,"Fabrication staff salaty, Analysis of the impo...","Health, Analysis of the market structure for t...","Education of the dealersвЂ™ sales staff, Quant...","Search and selection of dealers, Contracts of ...","Fabrication staff salaty, Analysis of the impo..."
4,1382,10667,4,5,5,5,193,54,49,152,...,1.788854,23.021729,1.140175,104.020671,1.516575,"Pick of contractors and providers, Engineers q...","Media-advertising, The best physical propertie...","Conduct customer analysis on the market, Firef...","Bicycles Production line 20000 units, Cumulati...","Pick of contractors and providers, Engineers q..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,1274,10563,5,4,5,4,184,10,1,11,...,1.673320,9.066422,0.547723,40.536403,1.095445,Quality of the purchased materials Raw (carbon...,Quantity of work shifts,"Development of productionвЂ™s optimisation, De...","Size of a reserve stock of FP, Employment of o...",Quality of the purchased materials Raw (carbon...
676,1526,10386,5,4,5,5,210,1,14,24,...,0.707107,7.797435,1.788854,23.114930,1.923538,Medium strategy of staff recruitment,Bonuses for innovation and rationalization of ...,Cancellation of bonus for violation of labor d...,Sending Engineers on training courses,Medium strategy of staff recruitment Bonuses f...
677,1670,10675,5,4,5,4,235,4,14,20,...,1.949359,9.833616,1.140175,26.159128,1.816590,"Optimisation of the loading unitвЂ™s shape, Th...","Size of a reserve stock of MR, Size of a reser...","Size of a reserve stock of MR, The volume of o...",The volume of of transportation - Large compan...,"Optimisation of the loading unitвЂ™s shape, Th..."
678,1808,10366,4,4,4,4,248,10,44,45,...,1.140175,13.527749,1.483240,58.909252,1.516575,Improvement of the documentation of entrance c...,Sending staff on short-term courses of creativ...,Conducting regular medical examinations and br...,"Dividend payout in cash, Transfer the rights t...",Improvement of the documentation of entrance c...


In [80]:
test

,team_id,id,game_id,Период_1_всего,Период_2_всего,Период_3_всего,Период_4_всего,Analytical thinking_prognoz,Systemic thinking_prognoz,Adaptability_prognoz,...,Категория 4 место стокл,Категория 5 оценка стокл,Категория 5 место стокл,Категория 6 оценка стокл,Категория 6 место стокл,period_1,period_2,period_3,period_4,TestSummAll
0,1912,10199,262,2,2,3,2,4,4,4,...,2.167948,12.481987,1.483240,47.700105,2.167948,"Setting the base selling price, Quantity of wo...","Additions to reserve capital, Volume discounts",Development and functioning of the companyвЂ™s...,"Hoverbikes Production line 2500 units, Setting...","Setting the base selling price, Quantity of wo..."
1,1458,10539,204,1,9,12,13,4,4,5,...,1.870829,21.158922,2.000000,54.892622,1.870829,Additions to reserve capital,"Pick of contractors and providers, Additional ...","[Company #1] Acquisition of the company, Metho...","Bicycles Production line 40000 units, Increase...",Additions to reserve capital Pick of contracto...
2,1348,10174,190,0,28,21,16,5,5,4,...,0.836660,16.195678,1.303840,68.240018,0.836660,Not decision,Improvement of the technology of entrance cont...,Establishment of mid-term contract - Industria...,"Quality of the purchased materials Lubricants""...",Not decision Improvement of the technology of ...
3,1760,10465,244,15,18,29,17,4,5,5,...,1.923538,12.597619,0.707107,40.419055,1.673320,Improvement of the technology of entrance cont...,R&D enhancement of assortment qualities Standa...,"To sell overdue accounts receivable, which do ...","Research on elasticity of market demand, [long...",Improvement of the technology of entrance cont...
4,1260,10066,183,7,23,16,6,5,4,4,...,1.673320,8.555700,1.673320,66.377707,1.643168,"Quantity of work shifts, Keep up the aesthetic...",Bonuses for the fulfillment of the production ...,"Development of productionвЂ™s optimisation, Qu...","Enguneers salary, Stepwise staff numerosityвЂ™...","Quantity of work shifts, Keep up the aesthetic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,1562,10433,214,15,6,1,0,4,5,4,...,1.303840,15.578832,1.095445,90.642154,1.949359,"Pick of contractors and providers, Value of pr...",Analysis of the importance of product properti...,Quantity of work shifts,Not decision,"Pick of contractors and providers, Value of pr..."
289,1278,10893,184,14,52,34,18,5,5,5,...,0.836660,13.638182,2.073644,74.888584,1.516575,R&D enhancement of assortment qualities Standa...,"Discipline, Moral qualities, Independently hir...",Collection information and statistics of the d...,"Creativity, Managers salary, Discipline, Hirin...",R&D enhancement of assortment qualities Standa...
290,1474,10909,206,0,0,0,1,3,4,3,...,1.414214,16.516658,0.547723,44.654227,1.923538,Not decision,Not decision,Not decision,The volume of of transportation - Worldwide co...,Not decision Not decision Not decision The vol...
291,1268,10889,184,6,2,5,1,3,3,3,...,1.483240,3.577709,0.547723,75.837326,1.516575,Renewal of checkpoints in production and stora...,"Security alarm upgrade, Evaluation of the char...","Advertising in print media, Radio advertising,...",Analysis of the product satisfaction,Renewal of checkpoints in production and stora...


In [81]:
train.columns

Index(['team_id', 'id', 'Analytical thinking', 'Systemic thinking',
       'Adaptability', 'Focus', 'game_id', 'Период_1_всего', 'Период_2_всего',
       'Период_3_всего', 'Период_4_всего', 'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz',
       'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 6 средняя оценка', 'Категория 6 среднее место',
       'Категория 1 оценка стокл', 'Категория 1 место стокл',
       'Категория 2 оценка стокл', 'Категория 2 место стокл',
       'Категория 3 оценка стокл', 'Категория 3 место стокл',
       'Категория 4 оценка стокл', 'Категория 4 место стокл',
       'Категория 5 оценка стокл', 'Категория 5 место стокл',
    

In [82]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['team_id', 'game_id',  'Systemic thinking_prognoz',
       'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 6 средняя оценка', 'Категория 6 среднее место', 'TestSummAll'
]]
y_train=train[['Systemic thinking']]


testnet=test[['team_id', 'game_id', 'Systemic thinking_prognoz',
       'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 6 средняя оценка', 'Категория 6 среднее место', 'TestSummAll'
]]

In [83]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   team_id                     680 non-null    int64  
 1   game_id                     680 non-null    int64  
 2   Systemic thinking_prognoz   680 non-null    int64  
 3   Категория 1 средняя оценка  680 non-null    float64
 4   Категория 1 среднее место   680 non-null    int64  
 5   Категория 2 оценка          680 non-null    float64
 6   Категория 2 среднее место   680 non-null    int64  
 7   Категория 3 средняя оценка  680 non-null    float64
 8   Категория 3 среднее место   680 non-null    int64  
 9   Категория 4 средняя оценка  680 non-null    float64
 10  Категория 4 среднее место   680 non-null    int64  
 11  Категория 5 средняя оценка  680 non-null    float64
 12  Категория 5 среднее место   680 non-null    int64  
 13  Категория 6 средняя оценка  680 non

In [84]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 34.0,
 3: 2.6666666666666665,
 4: 0.2769857433808554,
 5: 1.096774193548387,
 6: 13.6}

In [85]:
X_train,X_test,y_train,y_test = train_test_split(X_train, 
                                                 y_train, 
                                                 test_size=0.1, 
                                                 random_state=55, 
                                                 stratify=y_train)

In [86]:
#Обучаем 
pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id','Systemic thinking_prognoz','Категория 1 среднее место', 'Категория 2 среднее место', 'Категория 3 среднее место','Категория 4 среднее место','Категория 5 среднее место', 'Категория 6 среднее место'], text_features = ['TestSummAll'])
pool_test_solution = Pool(X_test,y_test, cat_features = ['team_id', 'game_id','Systemic thinking_prognoz','Категория 1 среднее место', 'Категория 2 среднее место', 'Категория 3 среднее место','Категория 4 среднее место','Категория 5 среднее место', 'Категория 6 среднее место'], text_features = ['TestSummAll'])
testnet_solution = Pool(testnet, cat_features = ['team_id', 'game_id','Systemic thinking_prognoz','Категория 1 среднее место', 'Категория 2 среднее место', 'Категория 3 среднее место','Категория 4 среднее место','Категория 5 среднее место', 'Категория 6 среднее место'], text_features = ['TestSummAll'])
#,'Категория 1 среднее место', 'Категория 2 среднее место', 'Категория 3 среднее место','Категория 4 среднее место','Категория 5 среднее место', 'Категория 6 среднее место'
model_test = CatBoostClassifier(iterations=250,  task_type='GPU',depth=12, verbose =True, random_seed=27, learning_rate=0.001,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='MultiClassOneVsAll', class_weights=class_weights)
model_test.fit(pool_train_solution,eval_set=pool_test_solution,verbose=True,early_stopping_rounds=50 )



0:	learn: 0.8743124	test: 0.4582403	best: 0.4582403 (0)	total: 333ms	remaining: 1m 22s
1:	learn: 0.9103863	test: 0.4531825	best: 0.4582403 (0)	total: 541ms	remaining: 1m 7s
2:	learn: 0.9308269	test: 0.4632981	best: 0.4632981 (2)	total: 738ms	remaining: 1m
3:	learn: 0.9386360	test: 0.5084108	best: 0.5084108 (3)	total: 957ms	remaining: 58.8s
4:	learn: 0.9405195	test: 0.7668661	best: 0.7668661 (4)	total: 1.15s	remaining: 56.4s
5:	learn: 0.9440457	test: 0.7618083	best: 0.7668661 (4)	total: 1.44s	remaining: 58.6s
6:	learn: 0.9409804	test: 0.7818357	best: 0.7818357 (6)	total: 1.67s	remaining: 58s
7:	learn: 0.9408511	test: 0.8018631	best: 0.8018631 (7)	total: 1.86s	remaining: 56.3s
8:	learn: 0.9417371	test: 0.8018631	best: 0.8018631 (7)	total: 2.06s	remaining: 55.3s
9:	learn: 0.9408511	test: 0.7818357	best: 0.8018631 (7)	total: 2.27s	remaining: 54.5s
10:	learn: 0.9387832	test: 0.8305297	best: 0.8305297 (10)	total: 2.47s	remaining: 53.7s
11:	learn: 0.9430482	test: 0.8305297	best: 0.8305297 (10

In [87]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)


In [88]:
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred, average='macro', zero_division=True))

Значение метрики recall на трейне :  0.9212908954596933


In [89]:
#Получение ответов
y_pred_solution = model_test.predict(testnet_solution)

In [90]:
# Посмотрим предварительный прогноз
test['Systemic thinking_prognoz'].value_counts()

4    140
5     93
3     54
1      4
6      2
Name: Systemic thinking_prognoz, dtype: int64

In [91]:
# Заводим результаты в тест
test['Systemic thinking']=y_pred_solution
test['Systemic thinking'].value_counts()

4    142
5     94
3     52
1      3
6      2
Name: Systemic thinking, dtype: int64

In [92]:
test_itog=test[['id','Systemic thinking']]
test_itog

,id,Systemic thinking
0,10199,4
1,10539,4
2,10174,5
3,10465,5
4,10066,4
...,...,...
288,10433,5
289,10893,5
290,10909,4
291,10889,4


In [93]:
test_itog.to_csv('прогноз по Systemic thinking предварительный второй этап и текст.csv', sep=',', index=None)